In [1]:
import sys
import os
sys.path.append('../')

In [13]:
import zarr
import numpy as np
import torch
import copy
import numcodecs

In [6]:
ep_dict = {
    'observation.image': [
        {'path': 'videos/observation.image_episode_000000.mp4', 'timestamp': 0.0},
        {'path': 'videos/observation.image_episode_000000.mp4', 'timestamp': 0.1}],
     'observation.state': torch.tensor([[152.0000, 398.0000], [157.1967, 399.2942]]),
     'action': torch.tensor([[169.5758, 402.3773], [190.2249, 404.4283]]),
     'episode_index': [0, 0],
     'frame_index': [0, 1],
     'timestamp': [0.0, 0.1],
     'next.reward': [0., 1.],
     'next.done': [False, True],
     'next.success': [False, True]
}

In [4]:
output_path = "outputs/eval/dataset_test/zarr_test"

root = zarr.open(output_path, mode='a')

In [14]:
def append_episode_to_zarr(ep_dict, zarr_group):
    for key, value in ep_dict.items():
        # 첫 번째 에피소드인 경우 해당 key로 데이터셋 생성
        if key not in zarr_group:
            data_shape = (0, *np.array(value).shape[1:])
            
            # object_codec 추가: 문자열 데이터를 저장하기 위해 JSON을 사용
            object_codec = numcodecs.JSON() if isinstance(value[0], dict) or isinstance(value[0], str) else None
            zarr_group.create_dataset(
                key,
                shape=data_shape,
                chunks=(1, *data_shape[1:]),
                dtype=np.array(value).dtype,
                maxshape=(None, *data_shape[1:]),
                object_codec=object_codec
            )
        
        # 데이터셋에 에피소드 데이터를 추가
        zarr_group[key].append(value)

In [15]:
for i in range(6):
    new_ep_dict = copy.deepcopy(ep_dict)
    new_ep_dict['episode_index'] = [i] * 2

    append_episode_to_zarr(new_ep_dict, root)

/home/sm/anaconda3/envs/lerobot/lib/python3.10/site-packages/zarr/creation.py:190: UserWarning: ignoring keyword argument 'maxshape'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)


In [16]:
zarr_data = zarr.open(output_path, mode='r')

In [18]:
list(zarr_data['observation.image'])

[{'path': 'videos/observation.image_episode_000000.mp4', 'timestamp': 0.0},
 {'path': 'videos/observation.image_episode_000000.mp4', 'timestamp': 0.1},
 {'path': 'videos/observation.image_episode_000000.mp4', 'timestamp': 0.0},
 {'path': 'videos/observation.image_episode_000000.mp4', 'timestamp': 0.1},
 {'path': 'videos/observation.image_episode_000000.mp4', 'timestamp': 0.0},
 {'path': 'videos/observation.image_episode_000000.mp4', 'timestamp': 0.1},
 {'path': 'videos/observation.image_episode_000000.mp4', 'timestamp': 0.0},
 {'path': 'videos/observation.image_episode_000000.mp4', 'timestamp': 0.1},
 {'path': 'videos/observation.image_episode_000000.mp4', 'timestamp': 0.0},
 {'path': 'videos/observation.image_episode_000000.mp4', 'timestamp': 0.1},
 {'path': 'videos/observation.image_episode_000000.mp4', 'timestamp': 0.0},
 {'path': 'videos/observation.image_episode_000000.mp4', 'timestamp': 0.1}]